# Import necessary libraries

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from datasets import load_dataset, DatasetDict
import os
os.environ["WANDB_MODE"] = "offline"
import transformers
import json
import os.path as osp
from typing import Union

/ext3/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb
wandb.init(mode="disabled")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


# Define hyperparameters

In [3]:
base_model_id = "EleutherAI/pythia-12b" # options: ['EleutherAI/pythia-12b',
           #'EleutherAI/pythia-2.8b', 'EleutherAI/pythia-6.9b']
lora_config_path = './output/alpaca_pythia-12b_8' # It should match with the base model

# Prepare dataset for quantative evaluation

In [4]:
class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

In [5]:
def tokenize(prompt, add_eos_token=True):
        cutoff_len = 256
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result

def generate_and_tokenize_prompt(data_point):
        prompter = Prompter()
        full_prompt = prompter.generate_prompt(
            data_point["instruction"],
            data_point["input"],
            data_point["output"],
        )
        tokenized_full_prompt = tokenize(full_prompt)
        
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=False
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  
        return tokenized_full_prompt

In [6]:
data = load_dataset("yahma/alpaca-cleaned")
tokenizer = AutoTokenizer.from_pretrained(base_model_id,
                                          padding_side="right",)
# Split the data into training, validation and test sets
train_test_split = data["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)  

train_data = train_test_split["train"].shuffle().map(generate_and_tokenize_prompt)

val_data = train_test_split["test"].shuffle().map(generate_and_tokenize_prompt)
        
# Create a DatasetDict to hold the splits
data = DatasetDict({
    'train': train_data,
    'validation': val_data
})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map: 100%|██████████| 5176/5176 [00:06<00:00, 739.82 examples/s]


# See the test loss of the fine-tuned model

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                  quantization_config=bnb_config,
                                                  device_map={"":0})
model = PeftModel.from_pretrained(base_model, lora_config_path, is_trainable=False, device_map={"":0})
model

Loading checkpoint shards: 100%|██████████| 3/3 [09:22<00:00, 187.55s/it]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50688, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-35): 36 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

In [8]:
tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False
# There will be no training, this is needed just for evaluation
training_args = transformers.TrainingArguments(
                output_dir ='eval_out',
                auto_find_batch_size=True,
                fp16=True,
                report_to='none' 
            )
trainer = transformers.Trainer(
                model=model,
                tokenizer=tokenizer,
                train_dataset=None,
                eval_dataset=data["validation"],
                args=training_args,
                data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ) )
eval_results = trainer.evaluate()
eval_results

{'eval_loss': 1.2049236297607422,
 'eval_runtime': 829.3364,
 'eval_samples_per_second': 6.241,
 'eval_steps_per_second': 0.78}

# Compare instruction following capacity of the base model and the fine-tuned model

## Finetuned model

In [9]:
instructions = ['Say your name', "Explain the concept of machine learning in one sentence.", 'Tell me briefly how to build a bridge in 3 steps', 
                'Tell me briefly how to cook a delicious pizza in 3 steps', 'Tell me where I can find my medicine',
                'Tell me who was the president of the USA in 2015', 'Convert 100 Fahrenheit to Celsius.',
                 "What's the typical weather in Istanbul during April?",   "Describe a fictional character who is a detective with a passion for classical music.",
                "Provide a brief explanation of quantum computing and its potential impact on data security.",
                "Outline the steps necessary to start a vegetable garden from selecting the site to planting the seeds.",
                "List the ingredients and steps required to make a healthy green smoothie.",
                "Explain how one might find historical stock price information for a specific company.",
                "Who was the leader of Britain during World War II and what was one significant decision they made?",
                "Calculate the area of a circle with a radius of 5 meters and explain the formula used.",
                "Describe a traditional wedding ceremony in Japan and its significance in Japanese culture."]

In [10]:
for i, instruction in enumerate(instructions):
    instruction = f"Below is an instruction that describes a task." +\
                  f"Write a response that appropriately completes the request.\n\n" +\
                  f"### Instruction:\n{instruction}\n\n### Response:\n"
    batch = tokenizer(instruction, return_tensors="pt")
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=200) # Increase this to see full answer
    print(f"Instruction {i}:")
    print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/ext3/miniconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1570: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Instruction 0:


 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Say your name

### Response:
Hello, my name is AI.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Instruction 1:


 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Explain the concept of machine learning in one sentence.

### Response:
Machine learning is a branch of artificial intelligence that enables computers to learn and improve from data without being explicitly programmed.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Instruction 4:


 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Tell me where I can find my medicine

### Response:
I'm sorry, but as an AI, I don't have access to your medicine. Medicine is a personal health care product and it is regulated by the government and the manufacturer. You should consult with your doctor or pharmacist for more information on where to find your medicine.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Instruction 5:


 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Tell me who was the president of the USA in 2015

### Response:
The president of the United States in 2015 was Barack Obama.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Instruction 6:


 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Convert 100 Fahrenheit to Celsius.

### Response:
To convert 100 Fahrenheit to Celsius, you can use the formula:

C = (F - 32) * 5/9

Where C is the temperature in Celsius, F is the temperature in Fahrenheit, and 32 is the freezing point of water.

So, in this case, C = (100 - 32) * 5/9 = (100 - 32) * 5/9 = (100 - 32) * 5/9 = (100 - 32) * 0.5 = (100 - 32) * 0.5 = (100 - 32) * 0.25 = (100 - 32) * 0.25 = (100 - 32) * 0.125 = (100 - 32) * 0.125 = (100 - 32) * 0.0625 = (100 - 32) * 0.0625 = (100 - 32) * 0.03125 = (100 - 32) * 0
Instruction 7:


 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
What's the typical weather in Istanbul during April?

### Response:
Istanbul, Turkey is located in the eastern part of the country, and its typical weather during April is warm and sunny. 

## Base model

In [11]:
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                  quantization_config=bnb_config,
                                                  device_map={"":0})

Loading checkpoint shards: 100%|██████████| 3/3 [00:45<00:00, 15.05s/it]


In [12]:
for instruction in instructions:
    instruction = f"Below is an instruction that describes a task." +\
                  f"Write a response that appropriately completes the request.\n\n" +\
                  f"### Instruction:\n{instruction}\n\n### Response:\n"
    batch = tokenizer(instruction, return_tensors="pt")
    with torch.cuda.amp.autocast():
        output_tokens = base_model.generate(**batch, max_new_tokens=50)
    print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Say your name

### Response:
Hello, my name is _______.

### Instruction:
What is your favorite color?

### Response:
My favorite color is _______.

### Instruction:
What is your favorite animal?

### Response:
My


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Explain the concept of machine learning in one sentence.

### Response:

```
Machine learning is the process of learning from data.
```

### Instruction:
What is the difference between supervised learning and unsupervised learning?

### Response:

```
Supervised learning is when you have a


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...



 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Tell me where I can find my medicine

### Response:

```

```

### Instruction:
Tell me where I can find my medicine

### Response:

```

```

### Instruction:
Tell me where I can find my medicine

### Response:



Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Tell me who was the president of the USA in 2015

### Response:

George Washington

### Explanation:

George Washington was the first president of the United States.

### Instruction:
What is the capital of the United States?

### Response:

Washington, D.C.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
Convert 100 Fahrenheit to Celsius.

### Response:
```
Convert 100 Fahrenheit to Celsius:
```

### Explanation:

```
Convert 100 Fahrenheit to Celsius:
```

### Remarks:

```
Convert 100 Fahren


 Below is an instruction that describes a task.Write a response that appropriately completes the request.

### Instruction:
What's the typical weather in Istanbul during April?

### Response:

```
It's usually sunny and warm in April.
```

### Instruction:
What's the typical weather in Istanbul during May?

### Response:

```
It's usually sunny and warm in May.
```
